<a href="https://colab.research.google.com/github/SaniyaBubere/Lead-Scoring-Model/blob/main/LeadScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The provided dataset includes information about leads generated for rental properties. Each row represents a lead and provides various details such as agent ID, lead status, lost reason, budget, lease duration, move-in date, source of lead, destination city and country, desired room type, and lead ID.

There are four rows in the dataset, and each row represents a lost lead. The lost reasons for the leads are "Not responding" and "Low budget". One lead has a specified budget of "£121 - £180 per week" and a lease duration of "Full Year Course Stay 40 - 44 weeks". The move-in date for one lead is "31/08/22".

The dataset also includes information about the source of the leads such as the source website, source city, source country, UTM source, and UTM medium. Finally, each lead has a unique lead ID.

In the context of sales and marketing, a lead is a potential customer or prospect who has shown interest in a company's products or services.

Lead status refers to the current stage of a lead in the sales process. It describes whether a lead is a potential customer, a lost opportunity, or a converted customer.


Lease refers to a contractual agreement between two parties, whereby the owner of a property (lessor) allows another party (lessee) to use the property for a specified period of time in exchange for rent

In [25]:
import pandas as pd
import numpy as np

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
df=pd.read_excel("/content/drive/MyDrive/Data_Science_Internship.xlsx")


In [28]:
df.head()

,Unnamed: 0,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,0.0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,1.0,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...
2,2.0,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,2022-08-31,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,3.0,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0.0,NaT,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
4,4.0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...


In [33]:
# Drop first column of dataframe
df = df.iloc[: , 1:]

#Data Inspection

In [34]:
# Get the number of rows and columns
rows, columns = df.shape

In [35]:
# Print the number of rows and columns
print("Number of rows: ", rows)
print("Number of columns: ", columns)

Number of rows:  46608
Number of columns:  15


In [36]:
# Dataset Duplicate Value Count
df[df.duplicated()]

,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
7604,96ab6152cb421b4d4012a4903c19004608a342a5f7c667...,WON,NaN,0-150,51.0,2022-09-10,226912ab42a1c1a4ff34aec12dcc58df1865371d40213f...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,44574c4ba2ea74ad4bf1e184133cdbf4e7390a3690beff...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,810c069040f6a9b16fdf976a901755508a87cb0224b169...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,578346e055c3a47dbf5d16afc9f2496efb30e0d15aa799...
11383,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,NaN,NaN,2022-08-08,b2b2a0ecb072ed25f1844a3325a810b85689bcc785ddb4...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,7f3fa48ca885678134842fa7456f3ece53a97f843b6101...,79f45ffcab6477ac462e7479564f3ce36f698316ba2372...,0fa4720e8f52f127b3eb09a73a2dc682a95eee72b5d2a9...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,355713b625fd46c7f8575307f87cc4d77f22695d0026d5...
14338,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,150.0,51.0,2022-09-01,146fb4ecbe78caa24102bbaac12e2559a8c8e32fb116d3...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,7f3fa48ca885678134842fa7456f3ece53a97f843b6101...,53f80da4f1f3ea38acb886dcb770b60dc8a7241b3e6810...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,d891a767b46652f2a86e2212c0ff5fa4f62d6a1664e361...
23688,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,120.0,51.0,2022-09-14,b2b2a0ecb072ed25f1844a3325a810b85689bcc785ddb4...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,7f3fa48ca885678134842fa7456f3ece53a97f843b6101...,3409d5d4bbb1e8b4c80ab0d47bb1f70ce475cfb3cd118c...,5e02dadaa7e4cd29809e2f8a115f1e5e9c01b29813a1fb...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,41e6c74985e2554aece9f841f8515b46182a02d8264390...
34306,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,NaN,0.0,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,4ed379d418bb86290a01117e9ceb0debffc4d1b7087db6...,5e02dadaa7e4cd29809e2f8a115f1e5e9c01b29813a1fb...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,4789356dfa2cab509afd747aacedd81425d18930e35a7a...
37457,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,NaN,NaN,2022-08-23,b2b2a0ecb072ed25f1844a3325a810b85689bcc785ddb4...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,7f3fa48ca885678134842fa7456f3ece53a97f843b6101...,efbfd394ac2dc6f05e74e167113f414b0fa50b64f82dfb...,13d4b96b6ee361b737e22404aeecca45f322ba7d4c2c42...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,4e4b409eaf076537510afbe2a56258304dc9473274d0df...
44564,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,NaN,0.0,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,4ed379d418bb86290a01117e9ceb0debffc4d1b7087db6...,a3a90dcfa0abf041f3e54b0dcc83fc052023cbe1f7c87a...,02bf1dfa9a0365a86223f0e4ac9eaa0517b06b2e927379...,NaN,0a406b69cdc3655b07d9feaff8636e1c86f85a2913f857...
44573,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,NaN,0.0,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,4ed379d418bb86290a01117e9ceb0debffc4d1b7087db6...,a3a90dcfa0abf041f3e54b0dcc83f

In [37]:
# Calculating the shape after removing duplicates
df = df.drop_duplicates(keep = 'first')
df.shape

(46598, 15)

In [38]:
# Dataset Info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46598 entries, 0 to 46607
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Agent_id        46598 non-null  object        
 1   status          46598 non-null  object        
 2   lost_reason     43235 non-null  object        
 3   budget          42905 non-null  object        
 4   lease           44259 non-null  object        
 5   movein          32965 non-null  datetime64[ns]
 6   source          46598 non-null  object        
 7   source_city     46598 non-null  object        
 8   source_country  46598 non-null  object        
 9   utm_source      46598 non-null  object        
 10  utm_medium      46598 non-null  object        
 11  des_city        46598 non-null  object        
 12  des_country     46598 non-null  object        
 13  room_type       23061 non-null  object        
 14  lead_id         46598 non-null  object        
dtypes:

In [39]:
df.describe()

<ipython-input-39-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
count,46598,46598,43235,42905,44259,32965,46598,46598,46598,46598,46598,46598,46598,23061,46598
unique,120,6,30,1840,311,477,683,4336,186,35,64,220,15,5,30574
top,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,LOST,Low availability,£60 - £120 Per week,Full Year Course Stay 40 - 44 weeks,2022-09-10 00:00:00,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,f542bdd16b93b6c8bfcdba9acf7d2d36e47ff378987f07...
freq,8177,43235,8883,9862,11111,2452,19306,8841,18812,31125,31761,7361,42669,12477,2
first,NaN,NaN,NaN,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2024-09-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Data Cleaning

In [40]:
df["status"].unique()

array(['LOST', 'WON', 'OPPORTUNITY', 'CONTACTED', 'PROCESSING',
       'IMPORTANT'], dtype=object)

In [41]:
df = df[df["status"].isin(["WON","LOST"])]

Replacing the given value with NA

In [42]:
value='9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0' 

In [43]:
count=df.isin([value]).sum()
count

Agent_id             0
status               0
lost_reason          0
budget               0
lease                0
movein               0
source            5946
source_city       8821
source_country    8612
utm_source          61
utm_medium        3183
des_city          2528
des_country       2528
room_type            0
lead_id              0
dtype: int64

In [44]:
# Replace missing values with a unique value
df.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan, inplace=True)

In [45]:
count=df.isin([value]).sum()
count

Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
movein            0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
room_type         0
lead_id           0
dtype: int64

In [46]:
df.isnull().sum()

Agent_id              0
status                0
lost_reason        3072
budget             3687
lease              2334
movein            13605
source             5946
source_city        8821
source_country     8612
utm_source           61
utm_medium         3183
des_city           2528
des_country        2528
room_type         23481
lead_id               0
dtype: int64

In [47]:
round(100*(df.isnull().sum()/len(df.index)), 2)

Agent_id           0.00
status             0.00
lost_reason        6.63
budget             7.96
lease              5.04
movein            29.38
source            12.84
source_city       19.05
source_country    18.60
utm_source         0.13
utm_medium         6.87
des_city           5.46
des_country        5.46
room_type         50.71
lead_id            0.00
dtype: float64

 Function to remove the columns having more than 40% missing values

In [48]:
pct_null = df.isnull().sum() / len(df)
missing_features = pct_null[pct_null > 0.40].index
df.drop(missing_features, axis=1, inplace=True)

In [49]:
round(100*(df.isnull().sum()/len(df.index)), 2)

Agent_id           0.00
status             0.00
lost_reason        6.63
budget             7.96
lease              5.04
movein            29.38
source            12.84
source_city       19.05
source_country    18.60
utm_source         0.13
utm_medium         6.87
des_city           5.46
des_country        5.46
lead_id            0.00
dtype: float64

1 column removed.



In [50]:
pct_null = df.isnull().sum() / len(df)
missing_features = pct_null[pct_null >= 0.70].index
df.drop(missing_features, axis=0, inplace=True)


In [51]:
df.shape

(46307, 14)

In [52]:
def get_all_unique_values(df):
    for col in df.columns:
        print(f"Unique values in column '{col}':")
        print(df[col].unique())
        print()

In [53]:
# Get and print all unique values
get_all_unique_values(df)

Unique values in column 'Agent_id':
['1deba9e96f404694373de9749ddd1ca8aa7bb823145a6fe3491dec605a526a92'
 '299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236b341962dec14cb4bb4'
 'c213697430c006013012dd2aca82dd9732aa0a1a6bca1371c92f18fd11d9bc5f'
 'eac9815a500f908736d303e23aa227f0957177b0e6756b915a3d3cc3a846b83b'
 '2306878a9ad9b57686cd623dd285aaa9b25afdf627f651af002888372e0c7d40'
 '44864c96fa1c36602f0d045b268981b6cab638a60fc207c89e3bba235f619450'
 'ab6bb4584e9946b135dca2e39d12abba3ea82d5ea927d05371cc49b0731261ca'
 '131127203c89e8219dbdfe2f597538759310f40918b22270401794e1300d2c9d'
 '2fca346db656187102ce806ac732e06a62df0dbb2829e511a770556d398e1a6e'
 'd4192f06768ab0f257c7f5e17ad021b075b995d4a186754a2ca3194ac77df31a'
 '50750ee66f27656c2b34d43078a064c3b9b8807938b6a312d8788698060e8517'
 'a9f80b4eaba3fd134bafafe7506e08940201964615f7eef25502c6eab6c85d82'
 'f1ece3b02f1e5989bb0918e468fbc3f3e60d74ed90809dd9767194e6e4d2960c'
 'bb22552c613cd3d30888fe8c4fecac110298f9f06698ef5daa88a7d41860b93c'
 '6b61e611d2

In [54]:
# Handle missing values by filling them with mode
df = df.fillna(df.mode().iloc[0])

In [55]:
df.isna().sum()

Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
movein            0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
lead_id           0
dtype: int64

In [62]:
# Convert all columns to categorical type
df = df.astype('category')

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46317 entries, 0 to 46607
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Unnamed: 0      46317 non-null  category
 1   Agent_id        46317 non-null  category
 2   status          46317 non-null  category
 3   lost_reason     43244 non-null  category
 4   budget          42623 non-null  category
 5   lease           43981 non-null  category
 6   movein          32707 non-null  category
 7   source          40366 non-null  category
 8   source_city     37486 non-null  category
 9   source_country  37695 non-null  category
 10  utm_source      46256 non-null  category
 11  utm_medium      43133 non-null  category
 12  des_city        43788 non-null  category
 13  des_country     43788 non-null  category
 14  room_type       22826 non-null  category
 15  lead_id         46317 non-null  category
dtypes: category(16)
memory usage: 4.5 MB


In [66]:
df.empty

False

# Feature Selection:


In [67]:
import seaborn as sns
import matplotlib.pyplot as plt